In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt
from time import time
from typing import List
from sklearn.model_selection import train_test_split
from typing import Dict
from sklearn.metrics import precision_recall_fscore_support


In [2]:
data = pd.read_csv('data/train.csv')
submission_test_data = pd.read_csv('data/test.csv')

trainval_data, test_data = train_test_split(data, test_size=0.2, stratify=data['feedback'])


In [55]:

def build_features(df: pd.DataFrame, categorical_features: list, ordinal_features: list, target_column: str):
    print('Starting feature engineering.')
    start_time = time()
    X = pd.DataFrame(columns=categorical_features + ordinal_features)
    X[categorical_features] = df[categorical_features]
    X[ordinal_features] = df[ordinal_features]
    if target_column != False:
        y = df[target_column]

    print(f'Built features. Took {time()-start_time:.2f} s.')
    if target_column != False:
        return X, y
    else:
        return X


categorical_features_binary = [
    'action_recommendation_id',
    'action_recommendation_type',
    'action_recommendation_category',
    'equipment_area',
    'usage_type',
    'equipment_category',
]
ordinal_features_binary =  [
    'speed_category',
    'load_category',
    'floors_category',
]


In [30]:
"""
Optimizes hyper-parameters for LightGBM using Optuna.
"""
import os

from sklearn.model_selection import train_test_split
import optuna
from dotenv import load_dotenv
from ngboost import NGBClassifier
from ngboost.distns import Bernoulli
from ngboost.scores import LogScore
from sklearn.tree import DecisionTreeClassifier
import click
from time import time

load_dotenv()

N_TRIALS = 10000

STORAGE = 'sqlite:///kone-binary-optuna.db'
STUDY_NAME_BINARY = 'kone-binary'
BETA = 1.3

def assign_categories_to_keep(data: pd.DataFrame, thresholds: dict) -> dict:
    tic = time()
    cats_to_keep = {}
    print(thresholds)
    for column_name in thresholds:
        print(f'Assigning categories to keep of {column_name} with threshold={thresholds[column_name]}')
        value_counts = data[column_name].value_counts(ascending=True)
        # Cumulative proportions of categories sorted from least frequent to most frequent.
        # For example if frequencies are: [1, 2, 4, 5, 10], then this produces [0.05, 0.14, 0.32, 0.55, 1.00]
        cat_portions_cumulative = value_counts.cumsum() / value_counts.sum()
        # Keep the most frequent categories via threshold.
        # In the above example, if threshold was 0.8, we would keep the categories with frequencies 4, 5 and 10
        # because 0.32 > (1 - 0.8). Doing it this way instead of using descending cumsum, we are guaranteed to keep
        # at least the proportion equal to threshold of data.
        cats_to_keep_mask = cat_portions_cumulative > (1 - thresholds[column_name])
        cats_to_keep[column_name] = cat_portions_cumulative.loc[cats_to_keep_mask].index
    toc = time()
    print(f'Assigning categories to keep took in total {toc-tic:.2f} seconds.')
    return cats_to_keep


def combine_rare_categories(data: pd.DataFrame, cats_to_keep: dict) -> pd.DataFrame:
        result = data.copy()
        tic = time()
        for column_name in cats_to_keep:
            print(f'Combining rare categories of {column_name}')
            print(f'old: {len(result[column_name].unique())}')
            combined_categories_label = 'uncommon category'
            cats_to_combine_mask = ~result[column_name].isin(cats_to_keep[column_name])
            result.loc[cats_to_combine_mask, column_name] = combined_categories_label
            print(f'new: {len(result[column_name].unique())}')
        toc = time()
        print(f'Combining took {toc - tic:.2f} seconds.')
        return result
    
    
def assign_categorical_features(x: pd.DataFrame, categorical_features: list):
    result = x.copy()
    if len(categorical_features) != 0:
        result[categorical_features] = result[categorical_features].astype('category')
    return result

class LgbOptimizer:

    def __init__(self, study_name: str, train_data):
        # We use only the train set in optimization. Later we split it to train and validation in each iteration.
        self.train_data = train_data
        sampler = optuna.samplers.NSGAIISampler()
        pruner = optuna.pruners.SuccessiveHalvingPruner()
        self.study = optuna.create_study(storage=STORAGE, study_name=study_name,
                                         direction='maximize', load_if_exists=True, sampler=sampler, pruner=pruner)

    def optimize(self):
        self.study.optimize(self._objective, n_trials=N_TRIALS, catch=(ValueError,), gc_after_trial=True)
        
    def _choose_categorical_thresholds(self, categorical_features: [str], trial: optuna.Trial):
        categorical_thresholds = {}
        for feature in categorical_features:
            categorical_thresholds.update({
                feature: trial.suggest_uniform(f'cat_threshold_{feature}', 0.5, 1.0)
            })
        return categorical_thresholds
    

    def _objective(self, trial: optuna.Trial):
        print('Splitting train and validation data.')
        start_time = time()
        train_data, val_data = train_test_split(self.train_data, test_size=0.2)
        print(f'Took {time()-start_time:.2f} s.')
        categorical_samples_threshold = trial.suggest_int('categorical_samples_threshold', 1, 1000)
        X_train, y_train = build_features(train_data, categorical_features_binary, ordinal_features_binary, 'feedback')
        X_val, y_val = build_features(val_data, categorical_features_binary, ordinal_features_binary, 'feedback')
        categorical_thresholds = self._choose_categorical_thresholds(categorical_features_binary, trial)
        cats_to_keep = assign_categories_to_keep(X_train, categorical_thresholds)
        
        X_train = combine_rare_categories(X_train, cats_to_keep)        
        X_val = combine_rare_categories(X_val, cats_to_keep)
        X_train = assign_categorical_features(X_train, categorical_features_binary)
        X_val = assign_categorical_features(X_val, categorical_features_binary)
        
        min_data_per_group = int(trial.suggest_discrete_uniform('min_data_per_group', 1, 1000, 1))
        cat_l2 = trial.suggest_uniform('cat_l2', 0.0, 100.0)
        cat_smooth = trial.suggest_uniform('cat_smooth', 0.0, 100.0)

        max_bin = int(trial.suggest_discrete_uniform('max_bin', 2, 10000, 1))
        min_data_in_bin = int(trial.suggest_discrete_uniform('min_data_in_bin', 1, 100, 1))
        subsample = trial.suggest_uniform('subsample', 0.0, 1.0)
        subsample_freq = int(round(trial.suggest_loguniform('subsample_freq', 1, 5000)))
        colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.0, 1.0)
        num_leaves = int(round(trial.suggest_loguniform('num_leaves', 2, 2000)))
        min_child_samples = int(trial.suggest_discrete_uniform('min_child_samples', 1, 1000, 1))
        reg_alpha = trial.suggest_uniform('reg_alpha', 0.0, 1.0)
        reg_lambda = trial.suggest_uniform('reg_lambda', 0.0, 1.0)
        learning_rate = trial.suggest_uniform('learning_rate', 0.001, 2.0)
        min_gain_to_split = trial.suggest_uniform('min_gain_to_split', 0.0, 2.0)
        early_stopping_rounds = int(trial.suggest_discrete_uniform('early_stopping_rounds', 1, 50, 1))
        params = {
            'min_data_per_group': min_data_per_group,
            'cat_l2': cat_l2,
            'cat_smooth': cat_smooth,
            'subsample': subsample,
            'subsample_freq': subsample_freq,
            'colsample_bytree': colsample_bytree,
            'num_leaves': num_leaves,
            'min_child_samples': min_child_samples,
            'reg_alpha': reg_alpha,
            'reg_lambda': reg_lambda,
            'learning_rate': learning_rate,
            'max_bin': max_bin,
            'min_data_in_bin': min_data_in_bin,
            'objective': 'binary',
            'min_gain_to_split': min_gain_to_split,
            'early_stopping_rounds': early_stopping_rounds,
        }
        
        print(f'Fitting LGB with parameters: {params}')
        start_time = time()
        
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, f'valid binary_error')

        classifier = lgb.LGBMClassifier(**params)
        classifier.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=100)
            
        print(f'Took {time()-start_time:.2f} s.')
        y_hat_prob = np.array(list(map(lambda x: x[1], classifier.predict_proba(X_val))))
        y_hat = y_hat_prob > trial.suggest_uniform('confidence_threshold', 0.5, 0.99)
        _, _, f2_score, _ = precision_recall_fscore_support(y_val, y_hat, beta=BETA, average='binary')
        return f2_score

trainval_data = pd.read_csv('data/train_data.csv')
LgbOptimizer(STUDY_NAME_BINARY, trainval_data).optimize()


[I 2021-11-20 22:44:17,318] A new study created in RDB with name: kone-binary


Splitting train and validation data.
Took 0.02 s.
Starting feature engineering.
Built features. Took 0.04 s.
Starting feature engineering.
Built features. Took 0.01 s.
{'action_recommendation_id': 0.7149126201146723, 'action_recommendation_type': 0.6463943401317294, 'action_recommendation_category': 0.6149990974080229, 'equipment_area': 0.8701220263797718, 'usage_type': 0.7583460208895939, 'equipment_category': 0.9065929923110392}
Assigning categories to keep of action_recommendation_id with threshold=0.7149126201146723
Assigning categories to keep of action_recommendation_type with threshold=0.6463943401317294
Assigning categories to keep of action_recommendation_category with threshold=0.6149990974080229
Assigning categories to keep of equipment_area with threshold=0.8701220263797718
Assigning categories to keep of usage_type with threshold=0.7583460208895939
Assigning categories to keep of equipment_category with threshold=0.9065929923110392
Assigning categories to keep took in tota

/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[I 2021-11-20 22:44:18,128] Trial 0 finished with value: 0.6264730737237407 and parameters: {'categorical_samples_threshold': 615, 'cat_threshold_action_recommendation_id': 0.7149126201146723, 'cat_threshold_action_recommendation_type': 0.6463943401317294, 'cat_

Early stopping, best iteration is:
[50]	training's binary_logloss: 0.322419	valid_1's binary_logloss: 0.336969
Took 0.25 s.
Splitting train and validation data.
Took 0.03 s.
Starting feature engineering.
Built features. Took 0.04 s.
Starting feature engineering.
Built features. Took 0.01 s.
{'action_recommendation_id': 0.8110706569994892, 'action_recommendation_type': 0.8289793302266073, 'action_recommendation_category': 0.9573538819049653, 'equipment_area': 0.5623166967042368, 'usage_type': 0.8124975017369271, 'equipment_category': 0.6593538715836443}
Assigning categories to keep of action_recommendation_id with threshold=0.8110706569994892
Assigning categories to keep of action_recommendation_type with threshold=0.8289793302266073
Assigning categories to keep of action_recommendation_category with threshold=0.9573538819049653
Assigning categories to keep of equipment_area with threshold=0.5623166967042368
Assigning categories to keep of usage_type with threshold=0.8124975017369271
As

/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[I 2021-11-20 22:44:19,032] Trial 1 finished with value: 0.9243994634800636 and parameters: {'categorical_samples_threshold': 200, 'cat_threshold_action_recommendation_id': 0.8110706569994892, 'cat_threshold_action_recommendation_type': 0.8289793302266073, 'cat_

...

Early stopping, best iteration is:
[28]	training's binary_logloss: 0.320413	valid_1's binary_logloss: 0.32496
Took 0.23 s.
Splitting train and validation data.
Took 0.02 s.
Starting feature engineering.
Built features. Took 0.03 s.
Starting feature engineering.
Built features. Took 0.01 s.
{'action_recommendation_id': 0.6980951755628022, 'action_recommendation_type': 0.9931665463025192, 'action_recommendation_category': 0.8007086178813452, 'equipment_area': 0.6145348342586441, 'usage_type': 0.9865869798053704, 'equipment_category': 0.7586090580922469}
Assigning categories to keep of action_recommendation_id with threshold=0.6980951755628022
Assigning categories to keep of action_recommendation_type with threshold=0.9931665463025192
Assigning categories to keep of action_recommendation_category with threshold=0.8007086178813452
Assigning categories to keep of equipment_area with threshold=0.6145348342586441
Assigning categories to keep of usage_type with threshold=0.9865869798053704
Ass

/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[I 2021-11-20 22:46:32,461] Trial 117 finished with value: 0.9491595690286562 and parameters: {'categorical_samples_threshold': 270, 'cat_threshold_action_recommendation_id': 0.6980951755628022, 'cat_threshold_action_recommendation_type': 0.9931665463025192, 'ca

[100]	training's binary_logloss: 0.336999	valid_1's binary_logloss: 0.334939
Did not meet early stopping. Best iteration is:
[98]	training's binary_logloss: 0.336923	valid_1's binary_logloss: 0.33477
Took 0.57 s.
Splitting train and validation data.
Took 0.03 s.
Starting feature engineering.
Built features. Took 0.05 s.
Starting feature engineering.
Built features. Took 0.01 s.
{'action_recommendation_id': 0.6887481723651272, 'action_recommendation_type': 0.7048875014335758, 'action_recommendation_category': 0.7627186167164024, 'equipment_area': 0.6622100899603495, 'usage_type': 0.7257467837142884, 'equipment_category': 0.8532236901796142}
Assigning categories to keep of action_recommendation_id with threshold=0.6887481723651272
Assigning categories to keep of action_recommendation_type with threshold=0.7048875014335758
Assigning categories to keep of action_recommendation_category with threshold=0.7627186167164024
Assigning categories to keep of equipment_area with threshold=0.6622100

/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


KeyboardInterrupt: 

In [31]:
params1 = {'cat_l2': 16.17929714582711, 'cat_smooth': 76.27366909954975, 'cat_threshold_action_recommendation_category': 0.860453204570097, 'cat_threshold_action_recommendation_id': 0.9713618866760998, 'cat_threshold_action_recommendation_type': 0.7902839124181956, 'cat_threshold_equipment_area': 0.6622100899603495, 'cat_threshold_equipment_category': 0.9976443081793678, 'cat_threshold_usage_type': 0.7485300842845735, 'categorical_samples_threshold': 158, 'colsample_bytree': 0.4047394265883333, 'confidence_threshold': 0.5272155481974249, 'early_stopping_rounds': 37.0, 'learning_rate': 0.37528737244069016, 'max_bin': 1195.0, 'min_child_samples': 221.0, 'min_data_in_bin': 76.0, 'min_data_per_group': 338.0, 'min_gain_to_split': 0.6081200602475842, 'num_leaves': 135.39248998095357, 'reg_alpha': 0.929229390150298, 'reg_lambda': 0.33865435836903246, 'subsample': 0.45849283413402686, 'subsample_freq': 5.055764599178599}

categorical_thresholds = {}
classifier_params = {}

int_params = [
    'min_data_per_group',
    'max_bin',
    'min_data_in_bin',
    'subsample_freq',
    'num_leaves',
    'min_child_samples',
    'early_stopping_rounds',
]
for key in params1.keys():
    prefix = 'cat_threshold_'
    if prefix in key:
        feature = key[len(prefix):]
        categorical_thresholds[feature] = params1[key]
    elif key in int_params:
        classifier_params[key] = int(round(params1[key]))
    elif (key == 'categorical_samples_threshold') or (key == 'confidence_threshold'):
        continue
    else:
        classifier_params[key] = params1[key]
classifier_params['objective'] = 'binary'
print(classifier_params)

test_data = pd.read_csv('data/test_data.csv')
train_data, val_data = train_test_split(trainval_data, test_size=0.2)

X_train, y_train = build_features(train_data, categorical_features_binary, ordinal_features_binary, 'feedback')
X_val, y_val = build_features(val_data, categorical_features_binary, ordinal_features_binary, 'feedback')
X_test, y_test = build_features(test_data, categorical_features_binary, ordinal_features_binary, 'feedback')

cats_to_keep = assign_categories_to_keep(X_train, categorical_thresholds)

X_train = combine_rare_categories(X_train, cats_to_keep)
X_train = assign_categorical_features(X_train, categorical_features_binary)

X_val = combine_rare_categories(X_val, cats_to_keep)
X_val = assign_categorical_features(X_val, categorical_features_binary)

X_test = combine_rare_categories(X_test, cats_to_keep)
X_test = assign_categorical_features(X_test, categorical_features_binary)

classifier = lgb.LGBMClassifier(**classifier_params)
classifier.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=100)

y_hat_prob = np.array(list(map(lambda x: x[1], classifier.predict_proba(X_test))))
y_hat = y_hat_prob > params1['confidence_threshold']
precision, recall, f2_score, support = precision_recall_fscore_support(y_test, y_hat, beta=2, average='binary')
       
print(precision, recall, f2_score, support)
print((y_hat == False).sum())
print((y_hat == True).sum())


{'cat_l2': 16.17929714582711, 'cat_smooth': 76.27366909954975, 'colsample_bytree': 0.4047394265883333, 'early_stopping_rounds': 37, 'learning_rate': 0.37528737244069016, 'max_bin': 1195, 'min_child_samples': 221, 'min_data_in_bin': 76, 'min_data_per_group': 338, 'min_gain_to_split': 0.6081200602475842, 'num_leaves': 135, 'reg_alpha': 0.929229390150298, 'reg_lambda': 0.33865435836903246, 'subsample': 0.45849283413402686, 'subsample_freq': 5, 'objective': 'binary'}
Starting feature engineering.
Built features. Took 0.03 s.
Starting feature engineering.
Built features. Took 0.01 s.
Starting feature engineering.
Built features. Took 0.01 s.
{'action_recommendation_category': 0.860453204570097, 'action_recommendation_id': 0.9713618866760998, 'action_recommendation_type': 0.7902839124181956, 'equipment_area': 0.6622100899603495, 'equipment_category': 0.9976443081793678, 'usage_type': 0.7485300842845735}
Assigning categories to keep of action_recommendation_category with threshold=0.860453204

/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[100]	training's binary_logloss: 0.290149	valid_1's binary_logloss: 0.297944
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.290149	valid_1's binary_logloss: 0.297944
0.8999277456647399 0.9831277750370004 0.9652793923893668 None
951
22144


In [54]:
train_data_all, val_data_all = train_test_split(data, test_size=0.2)

X_train_all, y_train_all = build_features(train_data_all, categorical_features_binary, ordinal_features_binary, 'feedback')
X_val_all, y_val_all = build_features(val_data_all, categorical_features_binary, ordinal_features_binary, 'feedback')
X_test_submit = build_features(submission_test_data, categorical_features_binary, ordinal_features_binary, False)

cats_to_keep = assign_categories_to_keep(X_train_all, categorical_thresholds)

X_train_all = combine_rare_categories(X_train_all, cats_to_keep)
X_train_all = assign_categorical_features(X_train_all, categorical_features_binary)

X_val_all = combine_rare_categories(X_val_all, cats_to_keep)
X_val_all = assign_categorical_features(X_val_all, categorical_features_binary)

X_test_submit = combine_rare_categories(X_test_submit, cats_to_keep)
X_test_submit = assign_categorical_features(X_test_submit, categorical_features_binary)

classifier_final = lgb.LGBMClassifier(**classifier_params)
classifier_final.fit(X_train_all, y_train_all, eval_set=[(X_train_all, y_train_all), (X_val_all, y_val_all)], verbose=100)
y_hat_prob_submit = np.array(list(map(lambda x: x[1], classifier_final.predict_proba(X_test_submit))))
y_hat_submit = y_hat_prob_submit > params1['confidence_threshold']
       
n_zeros_submit = ((y_hat_submit == False).sum())
n_ones_submit = ((y_hat_submit == True).sum())

print(f'Number of zeros in submit predictions: {n_zeros_submit}')
print(f'Number of ones in submit predictions: {n_ones_submit}')

submit_df = submission_test_data[['case_id', 'action_recommendation_id']]
submit_df.insert(2, 'feedback', y_hat_submit.astype(int))
display(submit_df)
submit_df.to_csv('data/submission.csv', index=False)


Starting feature engineering.
Built features. Took 0.04 s.
Starting feature engineering.
Built features. Took 0.01 s.
Starting feature engineering.
Built features. Took 0.01 s.
{'action_recommendation_category': 0.860453204570097, 'action_recommendation_id': 0.9713618866760998, 'action_recommendation_type': 0.7902839124181956, 'equipment_area': 0.6622100899603495, 'equipment_category': 0.9976443081793678, 'usage_type': 0.7485300842845735}
Assigning categories to keep of action_recommendation_category with threshold=0.860453204570097
Assigning categories to keep of action_recommendation_id with threshold=0.9713618866760998
Assigning categories to keep of action_recommendation_type with threshold=0.7902839124181956
Assigning categories to keep of equipment_area with threshold=0.6622100899603495
Assigning categories to keep of equipment_category with threshold=0.9976443081793678
Assigning categories to keep of usage_type with threshold=0.7485300842845735
Assigning categories to keep took 

/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/juho/.pyenv/versions/3.9.0/envs/exploration/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[100]	training's binary_logloss: 0.283579	valid_1's binary_logloss: 0.291618
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.283579	valid_1's binary_logloss: 0.291618
Number of zeros in submit predictions: 1088
Number of ones in submit predictions: 28340


,case_id,action_recommendation_id,feedback
0,9d54504e-c805-4859-b92e-a8df797323e7,ar00000250,1
1,554e89db-0d65-44f1-a3cb-d79662cd1058,ar00000188,1
2,6cb91017-5e91-446b-9064-409758334cea,ar00000005,1
3,496f3fe9-36a6-4b44-a686-5383584c4653,ar00000124,1
4,f8189755-b79e-4114-8a2b-84e08a2263f9,ar00000250,1
...,...,...,...
29423,62bf183c-eb55-4da1-b654-f245b320c6c0,ar00000082,1
29424,62bf183c-eb55-4da1-b654-f245b320c6c0,ar00000024,1
29425,e1cad254-5772-4f88-836c-03aa75f37341,ar00000209,1
29426,29ad636c-522c-46c0-abb8-e29cca030fea,ar00000174,1
